In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import datetime
import itertools as it

In [2]:
data = pd.read_csv('movie_bd_v5.csv')

# Предобработка

In [3]:
answers = {} #словарь для ответов

#расчет прибыли
def profit(revenue, budget):
    profit = revenue - budget
    return profit

#преобразование строк в списки 
data.genres = data.genres.apply(lambda x: x.split('|'))
data.director = data.director.apply(lambda x: x.split('|'))
data.cast = data.cast.apply(lambda x: x.split('|'))
data.production_companies = data.production_companies.apply(lambda x: x.split('|'))
data.release_date = data.release_date.apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y'))

#дополнительные колонки
data['profit'] = data.apply(lambda x: profit(x.revenue, x.budget), axis=1)
data['len_overview'] = data.overview.apply(lambda x:len(x.split(' ')))
data['len_title'] = data.original_title.apply(lambda x:len(x))
data['release_month'] = data.release_date.apply(lambda x: x.strftime('%B'))

#разбиения по различным столбцам
data_genre = data.copy()
data_genre = data_genre.explode('genres')
data_dir = data.copy()
data_dir = data_dir.explode('director')
data_cast = data.copy()
data_cast = data_cast.explode('cast')
data_company = data.copy()
data_company = data_company.explode('production_companies')

# 1. У какого фильма из списка самый большой бюджет?

In [4]:
a1 = data[data.budget == data.budget.max()].original_title
print(a1)
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object


# 2. Какой из фильмов самый длительный (в минутах)?

In [5]:
a2 = data[data.runtime == data.runtime.max()].original_title
print(a2)
answers['2'] = 'Gods and Generals (tt0279111)'

1157    Gods and Generals
Name: original_title, dtype: object


# 3. Какой из фильмов самый короткий (в минутах)?





In [6]:
a3 = data[data.runtime == data.runtime.min()].original_title
print(a3)
answers['3'] = 'Winnie the Pooh (tt1449283)'

768    Winnie the Pooh
Name: original_title, dtype: object


# 4. Какова средняя длительность фильмов?


In [7]:
a4 = data.runtime.mean()
print(a4)
answers['4'] = '110'

109.6585494970884


# 5. Каково медианное значение длительности фильмов? 

In [8]:
a5 = data.runtime.median()
print(a5)
answers['5'] = '107'

107.0


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [9]:
a6 = data[data.profit == data.profit.max()].original_title
print(a6)
answers['6'] = 'Avatar (tt0499549)'

239    Avatar
Name: original_title, dtype: object


# 7. Какой фильм самый убыточный? 

In [10]:
a7 = data[data.profit == data.profit.min()].original_title
print(a7)
answers['7'] = 'The Lone Ranger (tt1210819)'

1245    The Lone Ranger
Name: original_title, dtype: object


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [11]:
a8 = data[data.profit > 0].imdb_id.count()
print(a8)
answers['8'] = '1478'

1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [12]:
a9 = data[data.release_year == 2008].profit.max()
a9 = data[data.profit == a9].original_title
print(a9)
answers['9'] = 'The Dark Knight (tt0468569)'

599    The Dark Knight
Name: original_title, dtype: object


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [13]:
a10 = data[(2012 <= data.release_year)&(data.release_year <= 2014)].profit.min()
a10 = data[data.profit == a10].original_title
print(a10)
answers['10'] = 'The Lone Ranger (tt1210819)'

1245    The Lone Ranger
Name: original_title, dtype: object


# 11. Какого жанра фильмов больше всего?

In [14]:
a11 = data_genre.genres.value_counts().index[0]
print(a11)
answers['11'] = 'Drama'

Drama


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [15]:
a12 = data_genre[data_genre.profit > 0].groupby('genres').imdb_id.count().sort_values(ascending=False).index[0]
print(a12)
answers['12'] = 'Drama'

Drama


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [16]:
a13 = data_dir.groupby('director')['revenue'].sum().sort_values(ascending=False).index[0]
print(a13)
answers['13'] = 'Peter Jackson'

Peter Jackson


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [17]:
data_dir_gen = data_dir.copy()
data_dir_gen = data_dir_gen.explode('genres')
a14 = data_dir_gen[data_dir_gen.genres == 'Action'].director.value_counts().index[0]
print(a14)
answers['14'] = 'Robert Rodriguez'

Robert Rodriguez


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [18]:
a15 = data_cast[data_cast.release_year == 2012].groupby(['cast']).revenue.sum().sort_values(ascending = False).index[0]
print(a15)
answers['15'] = 'Chris Hemsworth'

Chris Hemsworth


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [19]:
a16 = data_cast[data_cast.budget > data_cast.budget.mean()].groupby('cast').imdb_id.count().sort_values(ascending=False).index[0]
print(a16)
answers['16'] = 'Matt Damon'


Matt Damon


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [20]:
data_cast_gen = data_cast.copy()
data_cast_gen = data_cast_gen.explode('genres')
a17 = data_cast_gen[data_cast_gen.cast == 'Nicolas Cage'].groupby('genres').imdb_id.count().sort_values(ascending=False).index[0]
print(a17)
answers['17'] = 'Action'

Action


# 18. Самый убыточный фильм от Paramount Pictures

In [21]:
a18 = data_company[data_company.production_companies == 'Paramount Pictures']
a18 = a18[a18.profit == a18.profit.min()].original_title
print(a18)
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

925    K-19: The Widowmaker
Name: original_title, dtype: object


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [22]:
a19 = data.groupby('release_year').revenue.sum().sort_values(ascending=False).index[0]
print(a19)
answers['19'] = '2015'

2015


# 20. Какой самый прибыльный год для студии Warner Bros?

In [23]:
a20 = data_company[data_company.production_companies.str.contains('Warner Bros')].groupby('release_year').profit.sum().sort_values(ascending = False).index[0]
print(a20)
answers['20'] = '2014'

2014


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [24]:
a21 = data.groupby('release_month').imdb_id.count().sort_values(ascending = False).index[0]
print(a21)
answers['21'] = 'Сентябрь'

September


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [25]:
a22 = data[(data.release_month == 'June')|(data.release_month == 'July')|(data.release_month == 'August')].imdb_id.count()
print(a22)
answers['22'] = '450'

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [26]:
a23 = data_dir[(data_dir.release_month == 'December')|(data_dir.release_month == 'January')|(data_dir.release_month == 'February')].groupby('director').imdb_id.count().sort_values(ascending=False).index[0]
print(a23)
answers['23'] = 'Peter Jackson'

Peter Jackson


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [27]:
a24 = data_company.groupby('production_companies').len_title.mean().sort_values(ascending=False).index[0]
print(a24)
answers['24'] = 'Four By Two Productions'

Four By Two Productions


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [28]:
a25 = data_company.groupby('production_companies').len_overview.mean().sort_values(ascending=False).index[0]
print(a25)
answers['25'] = 'Midnight Picture Show'

Midnight Picture Show


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [29]:
a26 = data[data.vote_average > np.percentile(data.vote_average, 99)].original_title
print(a26)
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [33]:
a27 = Counter()
for i in range(0,len(data)):
    actors = data.cast[i]
    for j in list(it.combinations(actors, 2)):
        if j not in a27:
            a27[j] = 1 
        else:
            a27[j] += 1
a27.most_common(1)
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

# Submission

In [34]:
answers #проверка ответов

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}